In [15]:
import sys
sys.path.insert(0, "../")
import numpy as np
import pandas as pd
from src.utils.utils import create_dataset
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [16]:
!ls 

1.0-eda.ipynb	       3.0-inference.ipynb	      submission.csv
2.0-train_split.ipynb  4.0-create-zarr-dataset.ipynb
256x256-images.ipynb   oof_predicts


In [17]:
INPUT_PATH = Path("/hdd/kaggle/hubmap/input_v2")
PATH_FOR_CROPS = "/hdd/kaggle/hubmap/input_v2/train_v1_1024"
df_train = pd.read_csv(INPUT_PATH / "train.csv")
train_info = pd.read_csv(INPUT_PATH / "HuBMAP-20-dataset_information.csv")
train_tiffs = list((INPUT_PATH / "train").glob('*.tiff'))

## 5 fold multilabel stratified split

In [18]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [19]:
def manual_bin(square):
    if square == 0:
        return 0
    elif 0 < square <=0.1:
        return 1
    elif 0.1 < square <= 0.2:
        return 2
    elif 0.2 < square <= 0.3:
        return 3
    elif 0.3 < square <= 0.4:
        return 4
    elif 0.4 < square <= 0.6:
        return 5
    elif 0.6 < square <= 0.8:
        return 6
    elif 0.8 < square:
        return 7
    
# df['square'] = df['square']/512**2

In [20]:
!ls ../../input/train_1024_256

images	masks  meta.csv  split_v1.csv


In [21]:
df = pd.read_csv("../../input/train_1024_256/meta.csv")

In [22]:
df.columns

Index(['img_id', 'glomerulus_pix', 'medulla', 'cortex', 'outer_stripe',
       'Inner medulla', 'Outer Medulla', 'file'],
      dtype='object')

In [23]:
bining_cols = ['glomerulus_pix', 'medulla', 'cortex', 'outer_stripe',
       'Inner medulla', 'Outer Medulla']
for col in bining_cols:
    df[col] = df[col]/256**2
    df[col] = df[col].apply(manual_bin)

In [24]:
df.head()

,img_id,glomerulus_pix,medulla,cortex,outer_stripe,Inner medulla,Outer Medulla,file
0,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...
1,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...
2,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...
3,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...
4,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...


In [25]:
df["glomerulus_pix"].value_counts()

0    5887
7    2380
6     316
5     311
1     263
2     164
4     153
3     132
Name: glomerulus_pix, dtype: int64

In [16]:
df['fold'] = 0

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold_n, (train_index, test_index) in enumerate(mskf.split(df, df.iloc[:, :7])):
    df.loc[test_index, 'fold'] = fold_n

/home/trytolose/miniconda3/envs/audio/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [18]:
df.to_csv("/hdd/kaggle/hubmap/input_v2/train_v1_1024/split_v1.csv")

## Group KFold split

In [28]:
df["fold"] = 0
ids = df['img_id'].unique()
d = df['img_id'].value_counts()
split = [
    [0, 10, 9],
    [1, 7, 14],
    [8, 13, 2],
    [6, 3, 12],
    [4, 5, 11],
]
for fold, s in enumerate(split):
    img_ids = d[s].index.tolist()
    df.loc[df['img_id'].isin(img_ids), "fold"] = fold

In [29]:
for i in range(5):
    print(df[df['fold']==i]['img_id'].unique())

['2f6ecfcdf' '0486052bb' '4ef6695ce']
['aaa6a05cc' '095bf7a1f' 'c68fe75ea']
['b2dc8411c' 'afa5e8098' '1e2425f28']
['8242609fa' 'cb2d976f4' '54f2eec69']
['b9a3865fc' 'e79de561c' '26dc41664']


In [13]:
!ls ../../input/train_v3_4096_1024

images	masks  meta.csv  split_v1.csv


In [21]:
!ls ../../input

train_v3_4096_1024  train_v4_4096_1024	zarr_pdf  zarr_train  zarr_train_orig


In [14]:
df.to_csv("../../input/train_1024_256/split_v1.csv")

In [83]:
# !rm -rf /hdd/kaggle/hubmap/input_v2/train_v1_1024/

In [25]:
ids = df['img_id'].unique()
d = df['img_id'].value_counts()
d

4ef6695ce    1220
c68fe75ea    1175
afa5e8098    1130
cb2d976f4     893
26dc41664     781
b9a3865fc     774
8242609fa     760
095bf7a1f     657
1e2425f28     590
0486052bb     353
2f6ecfcdf     321
e79de561c     313
54f2eec69     304
b2dc8411c     207
aaa6a05cc     128
Name: img_id, dtype: int64

In [69]:
std_min = 1000
x_min = None
for i in tqdm(range(1000000)):
    x = np.random.choice(np.arange(15), size=(5, 3), replace=False)
    m = []
    for i in range(5):
#         print(d[x[i]].sum())
        m.append(d[x[i]].sum())
    mm = np.std(m)
    if mm < std_min:
        std_min = mm
        x_min = x.copy()

100%|██████████| 1000000/1000000 [12:11<00:00, 1366.35it/s]


In [70]:
print(x_min)
for i in range(5):
    print(d[x_min[i]].sum())

[[ 0 10  9]
 [ 1  7 14]
 [ 8 13  2]
 [ 6  3 12]
 [ 4  5 11]]
1894
1960
1927
1957
1868


In [84]:
std_min

35.75136361035757

## Sampling test

In [30]:
FOLD_IMGS = {
    0: ["4ef6695ce", "0486052bb", "2f6ecfcdf"],
    1: ["c68fe75ea", "095bf7a1f", "aaa6a05cc"],
    2: ["afa5e8098", "1e2425f28", "b2dc8411c"],
    3: ["cb2d976f4", "8242609fa", "54f2eec69"],
    4: ["26dc41664", "b9a3865fc", "e79de561c"],
}
def manual_bin(square):
    if square == 0:
        return 0
    elif 0 < square <=0.1:
        return 1
    elif 0.1 < square <= 0.2:
        return 2
    elif 0.2 < square <= 0.3:
        return 3
    elif 0.3 < square <= 0.4:
        return 4
    elif 0.4 < square <= 0.6:
        return 5
    elif 0.6 < square <= 0.8:
        return 6
    elif 0.8 < square:
        return 7

def create_dataset_annotation(df, img_size):
    bining_cols = ['glomerulus_pix', 'medulla', 'cortex', 'outer_stripe',
           'Inner medulla', 'Outer Medulla']
    for col in bining_cols:
        df[col] = df[col]/img_size**2
        df[col] = df[col].apply(manual_bin)
    return df

In [31]:
df = create_dataset_annotation("../../input/train_1024_256/meta.csv")
df.head()

,img_id,glomerulus_pix,medulla,cortex,outer_stripe,Inner medulla,Outer Medulla,file,fold
0,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
1,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
2,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
3,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
4,2f6ecfcdf,0,7,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0


In [36]:
x = Path("../../input/train_1024_256/meta.csv")
x.parent

PosixPath('../../input/train_1024_256')